# Ablation Study Example Notebook
 
This notebook provides an example workflow for generating comparative plots from RL training experiments in the CoRL framework. This includes training, evaluating and plotting policy performance data. This example will use the safe-autonomy-sims' 3D Satellite Inspection environment.
 
Author: John McCarroll

## Step 1 - Training
 
We will first need to train a couple policy networks to evaluate. Below, we will walk through how to configure and run a single training job or trial, for evaluation. We will use the base Inspection environment (with PPO and no Runtime Assurance), but configure training to run with different seeds to evaluate the effect of random initialization on the learned policy.

### Step 1a - Setting Seeds
 
Navigate to the Inspection environment's task config: 'safe-autonomy-sims/configs/tasks/cwh3d_inspection/cwh3d_task.yml'. Find the `env_config_updates` dict definition (line 24) and add a new item underneath named 'seed'. It should look something like this:
 
``` 
env_config_updates: &env_config_updates
 TrialName: CWH-3D-INSPECTION
 output_path: /tmp/safe_autonomy/
 seed: 11
```
 
For this example, I will be comparing the seeds 11 and 843.
 
 
 
### Step 1b - Configuring Checkpoints
 
Evaluation data will be generated AFTER training. This means it is crucial to save out policy checkpoints during training. Using the same task config file from before, let's navigate to the `tune_config_updates` dict definition (~line 29). We are primarily concerned with the `checkpoint_freq` (the number of iterations between saved checkpoints) and `keep_checkpoints_num` (the maximum number of checkpoints to save) variables. Here's an example configuration:
 
```
tune_config_updates: &tune_config_updates
 local_dir: /tmp/safe_autonomy/ray_results/
 checkpoint_freq: 5
 keep_checkpoints_num: 400
 stop:
   training_iteration: 50
```
 
Notes:
   - The 'stop' subdictionary defines training termination criteria. Above, training is configured to terminate after 50 iterations.
   - The string given to the 'local_dir' key is the path at which our training output (and checkpoints) will be saved. Tune will create a new directory with a long name consisting of the TrialName (defined in the environment config), hyperparameter values, and a timestamp. This directory will hold the checkpoints needed for evaluation. We will need to aggregate our experiment output directory paths into a single dictionary in a later step.
 
 
 
### Step 1c - Running Training
 
All that is left in this portion is to run CoRL's train_rl.py script. For that we need an experiment config. The Inspection environment's experiment config is located at 'safe-autonomy-sims/configs/experiments/inspection/inspection_3d.yml'. Ensure that this experiment config contains the path to the task config file we modified in the previous step.
 
To launch a training job, open a terminal and navigate to the root of your local safe-autonomy-sims repository. Then, enter the command found below:
 
```shell
$ python -m corl.train_rl --cfg /path/to/inspection/experiment/config
```

## Step 2 - Evaluate Trained Policies
 
Now that we have trained a couple of policies, let's set up to run CoRL's Evaluation Framework in order to generate data for our comparative plot. We will need a PlatformSerializer class, the experiment and task config file paths, a dictionary of experiment names to training output folder paths KVPs, a plot output path, and a plot config dictionary.

### Step 2a - PlatformSerializer
 
The Evaluation Framework makes use of a PlatformSerializer class to save and load platform state. These PlatformSerializer classes are specific to the Platforms used in our experiment. We can create a PlatformSerializer for a custom Platform by extending the corl.evaluation.runners.section_factories.plugins.platform_serializer.PlatformSerializer class. Thankfully, we have a PlatformSerializer class prepared for our Inspection environment. We'll import it below, along with the run_ablation_study method we will use.

In [ ]:
from saferl.evaluation.evaluation_api import run_ablation_study

# import our serializer for CWH platforms
from saferl.evaluation.launch.serialize_cwh3d import SerializeCWH3D

### Step 2b - Defining Variables
 
Now that we have trained a few policies, it's time to aggregate and evaluate that data. Let's begin defining some arguments needed to run our ablation study. We will need to define the paths for our training job's experiment config and task config. We will also define the path where we want our comparative (sample complexity) plot to be saved. Aside from simple paths, there are four key arguments to define, which determine much of the function of the evaluation framework. Below, we discuss the details of each argument.
 
 
#### Training Output Directories
Create a dictionary of "experiment name" to "training output directory path" KVPs. The keys will be used as labels on the generated plot, so feel free to give them semantic meaning to your experiment. See the 'training_output_dirs' variable below for an example. These output directories will be searched for checkpoint subdirectories. policies will be loaded from any found checkpoints and used to run evaluation episodes.
 
 
#### Plot Config
This dictionary defines the kwargs passed to seaborn's lineplot method to create the sample complexity plot. Only two kwargs are required: 'x_axis' and 'y_axis'. The 'x_axis' variable has three supported units of training time: 'num_interactions', 'num_episodes', and 'iterations'. The 'y_axis' variable is a bit more flexible, as it can be assigned to the name of any Metric found in the metrics_config (as long as the Metric has a numeric value). This allows users to create comparative training plots of custom Metrics values (ex. safety violations or runtime assurance interventions). For this example, we will use the TotalReward metric.
 
Other kwargs can include anything accepted by seaborn's lineplot method. Take 'hue', for example, which groups experiment data together based on categorical data (such as the 'experiment' name column). Another kwarg worth mentioning is 'ci', which stands for 'confidence interval'. This kwarg sets the statistical 'error bound' visualized on the resulting plot. This variable can be an integer (as in 'confidence interval' of 90 percent), but by default 'ci' is assigned the value 'sd' (which stands for one 'standard deviation').
 
 
#### Metrics Config
This dict defines the scope, name, class path, and config of every Metric to be calculated during evaluation episodes. The metrics_config has two 'scopes': 'world' and 'agent'. Metrics defined in the 'world' scope apply to the generic Episode data, whereas metrics defined in the 'agent' scope apply specifically to data available to agents. The 'agent' key is assigned a dictionary of lists. The keys are either a specific agent's name or '__default__' (if we want the Metric applied to all agents). The lists contain dictionary definitions of each Metric, which must include a 'name', 'functor', and 'config'. See the cell below for a simple example metric_config, defining only the TotalReward Metric in the default agent scope.
The nested structure of the metrics_config is based off of the yaml config file used in CoRL's evaluation framework. If it is difficult to follow, the usage guide, found (here)[https://github.com/act3-ace/act3-rl/corl/-/blob/main/docs/evaluation_framework/user_guide.md#521-write-the-metrics-configuration-file], contains more details on Metrics configuration.
 
 
#### TestCaseManager Config
The TestCaseManager is a class behind the scenes of the evaluation framework responsible for coordinating the initial parameter values used in each evaluation episode or "test case". This config is not required. If a test_case_manager_config kwarg is not defined for the run_ablation_study or evaluate methods, then the task's default ParameterProviders will be used to initialize the environment state for each evaluation episode. If a specific TestCaseStrategy is desired to define an explicit set of test cases for each policy evaluation, then one only needs to define two items in the test_case_manager_config dict. The first, 'test_case_strategy_class_path', is the class path to the TestCaseStrategy desired for the evaluation. The second item is 'config', which simply defines the kwargs passed to the TestCaseStrategy's constructor.
 
Below, the default test_case_manager_config is redundantly defined, for example purposes.

In [ ]:
# Define variables

training_output_dirs = {
    'experiment_name1': '/absolute/path/to/training/output/directory',
    'experiment_name2': '/absolute/path/to/training/output/directory2',
}
expr_config = '/absolute/path/to/experiment/config/file.yml'
task_config_path = '/absolute/path/to/task/config/file.yml'
plot_ouput = '/absolute/path/to/desired/plot/save/location.png'


plot_config = {
    "y_axis": "TotalReward",
    "x_axis": "num_iteration"
}

metrics_config = {
    "agent": {
        "__default__": [
            {
                "name": "TotalReward",
                "functor": "corl.evaluation.metrics.generators.rewards.TotalReward",
                "config": {
                    "description": "total reward calculated from test case rollout"
                }
            },
        ]
    }
}

default_test_case_manager_config = {
    "test_case_strategy_class_path": "corl.evaluation.runners.section_factories.test_cases.default_strategy.DefaultStrategy",
    "config": {
        "num_test_cases": 3
    }
}

## Step 3 - Analyze Data
 
The only thing left to do is launch the evaluation and generate plots. We'll use the run_ablation_study method, which sequentially loads each checkpoint policy, executes the defined evaluation episodes with said policy, calculates Metrics, organizes the data into a pandas.DataFrame, and saves out a seaborn plot to the specified location. It also returns the DataFrame, in case the user wants to further analyze the collected data.

In [ ]:
# launch ablation study pipeline
dataframe = run_ablation_study(training_output_dirs, task_config_path, expr_config, metrics_config, SerializeCWH3D, test_case_manager_config=default_test_case_manager_config, plot_output_path=plot_ouput)